In [3]:
%load_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
import pandas as pd

from utils.loading import load_paths_and_graph

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
paths_and_graph = load_paths_and_graph()

loading raw data from tsv files...
formatting data...
extracting additional information from source code...


In [5]:
articles = paths_and_graph["articles"]
articles.sort_values(by="num_links", ascending=False)

,article,article_decoded,links,num_links
3920,Sudan,Sudan,"[(2007 Schools Wikipedia Selection, index, ind...",606
1433,Europe,Europe,"[(2007 Schools Wikipedia Selection, index, ind...",558
2504,List_of_countries,List_of_countries,"[(2007 Schools Wikipedia Selection, index, ind...",558
1694,Germany,Germany,"[(2007 Schools Wikipedia Selection, index, ind...",552
2863,Mozambique,Mozambique,"[(2007 Schools Wikipedia Selection, index, ind...",550
...,...,...,...,...
4273,U.S._Open_%28tennis%29,U.S._Open_(tennis),NaN,0
4274,U.S._Route_66,U.S._Route_66,NaN,0
1541,Floyd_Mayweather_Jr.,Floyd_Mayweather_Jr.,NaN,0
345,Arsenal_F.C.,Arsenal_F.C.,NaN,0


In [10]:
articles[articles.num_links == 0].links

22      NaN
63      NaN
345     NaN
738     NaN
807     NaN
       ... 
4287    NaN
4391    NaN
4411    NaN
4417    NaN
4501    NaN
Name: links, Length: 63, dtype: object

In [12]:
hyperlinks = pd.concat(
    list(map(
        lambda row: pd.DataFrame(
            data={"from": [row.article_decoded]*row.num_links, 
            "to": [link[2] for link in row.links]}
        ) if row.num_links > 0 else pd.DataFrame(), 
        articles.itertuples(index=False)
    ))
)

hyperlinks.drop_duplicates(inplace=True)
hyperlinks

,from,to
0,Áedán_mac_Gabráin,index
1,Áedán_mac_Gabráin,subject.History.British_History.British_Histor...
2,Áedán_mac_Gabráin,subject.People.Historical_figures
3,Áedán_mac_Gabráin,4250.png
5,Áedán_mac_Gabráin,D%C3%A1l_Riata
...,...,...
16,Zulu,Cape_Town
17,Zulu,AK-47
18,Zulu,Ladysmith_Black_Mambazo
23,Zulu,Wikipedia_Text_of_the_GNU_Free_Documentation_L...


In [13]:
hyperlinks.groupby(by="from").count().sort_values("to", ascending=False).head(5)

,to
from,
List_of_countries,491
List_of_circulating_currencies,471
List_of_sovereign_states,423
United_States,334
Africa,288


In [14]:
hyperlinks.groupby(by="to").count().sort_values("from", ascending=False).head(15)

,from
to,
index,4536
Wikipedia_Text_of_the_GNU_Free_Documentation_License,4536
disclaimer,4536
United_States,1519
United_Kingdom,962
France,947
Europe,919
England,734
Germany,733


In [15]:
hyperlinks.query("`from` == `to`")

,from,to
24,American_Revolutionary_War,American_Revolutionary_War
10,Athens,Athens
63,Autostereogram,Autostereogram
42,Baltic_Sea,Baltic_Sea
13,Beijing,Beijing
...,...,...
10,War,War
4,Wikimedia_Foundation,Wikimedia_Foundation
3,Wikipedia_Text_of_the_GNU_Free_Documentation_L...,Wikipedia_Text_of_the_GNU_Free_Documentation_L...
84,World_Trade_Organization,World_Trade_Organization


In [18]:
G = nx.DiGraph()

G.add_nodes_from(articles.article_decoded)
G.add_edges_from(hyperlinks.values)

In [33]:
# sanity check the graph

distance_matrix = paths_and_graph["shortest-path-distance-matrix"]

reachable_values = np.argwhere(~np.isnan(distance_matrix))
sampled_indices = reachable_values[np.random.choice(len(reachable_values), size=40, replace=False)]

for source_index, target_index in sampled_indices:
    source = articles.article_decoded.iloc[source_index]
    target = articles.article_decoded.iloc[target_index]
    
    found_shortest_length = float("inf")
    try:
        shortest_path = nx.shortest_path(G, source=source, target=target)
        found_shortest_length = len(shortest_path)
    finally:
        assert distance_matrix[(source_index, target_index)] == found_shortest_length, f"Mismatch for {source} → {target}"

AssertionError: Mismatch for Imagism → Oceania